In [1]:
import sys

sys.path.append("lib")

from functorch import vmap

import matplotlib.pyplot as plt

import lib
import torch
import math
import numpy

In [2]:
device = torch.device("cuda")

In [3]:
Nx = 10
Ny = 10
mesh = lib.mesh.RectangleMesh({"xmin": 0, "xmax": 1, "ymin": 0, "ymax": 1, "Nx": Nx, "Ny": Ny}, device)
left = torch.isclose(mesh.nodes[:,0], torch.zeros(1, device=device)).nonzero().squeeze(1)
right = torch.isclose(mesh.nodes[:,0], torch.ones(1, device=device)).nonzero().squeeze(1)
bottom = torch.isclose(mesh.nodes[:,1], torch.zeros(1, device=device)).nonzero().squeeze(1)
top = torch.isclose(mesh.nodes[:,1], torch.ones(1, device=device)).nonzero().squeeze(1)

In [4]:
problem = lib.problem.FEProblem(mesh)
problem.add_variable("ux", lambda x : torch.sin(x[0]))
problem.add_variable("uy", lambda x : torch.sin(x[1]))
problem.add_weakform(lib.weakform.Diffusion(problem, {"variable":"ux"}))
problem.add_weakform(lib.weakform.Diffusion(problem, {"variable":"uy"}))
problem.init_solution()

In [5]:
problem.add_nodal_bc(lib.bc.DirichletBC(problem, {"value":0,"boundary":left,"variable":"ux"}))
problem.add_nodal_bc(lib.bc.DirichletBC(problem, {"value":0.1,"boundary":right,"variable":"ux"}))
problem.add_nodal_bc(lib.bc.DirichletBC(problem, {"value":-0.1,"boundary":bottom,"variable":"uy"}))
problem.add_nodal_bc(lib.bc.DirichletBC(problem, {"value":0.1,"boundary":top,"variable":"uy"}))

In [6]:
problem.solve()

Iteration 0, ||r|| = 1.292E+00
Iteration 1, ||r|| = 2.078E-16


In [7]:
problem.write_vtk("foo.vtk")